# 라이브러리 추가

In [1]:
# 기초 전처리
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 캔버스 사이즈 적용
plt.rcParams['figure.figsize'] = (12, 9)

# 컬럼 전체 확인 가능하도록 출력 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

# pandas 결과값의 표현 범위 소수점 2자리수로 변경
pd.options.display.float_format = '{:.2f}'.format

# 파일 로드위한 directory 확인 및 현재 경로로 설정
a = os.getcwd()
os.chdir(a)

In [2]:
import MeCab # 윈도우 명령어
import re

mecab = MeCab.Tagger()

def mecab_nouns(text):
    nouns = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 POS가 명사 분류에 속하는 토큰만 선택.
    for token in temp:
        if token[1] == "NNG" or token[1] == "NNP" or token[1] == "NNB" or token[1] == "NNBC" or token[1] == "NP" or token[1] == "NR":
            nouns.append(token[0])
        
    return nouns

def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append(token[0])
    
    return morphs

def mecab_pos(text):
    pos = []
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    pos = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    return pos

In [3]:
import re

def message_cleaning(docs):

    """
        1. Photo, Emoticon은 내용알기 어려움 -> 제거
        
        2. 자음/모음 표현 처리방법.
            1) "ㅇㅇ" ,"ㅋㅋㅋㅋㅋ" 같은 자음만 존재하는 표현이나, "ㅡㅡ", "ㅠㅠ" 같은 모음만 존재하는 표현들은
            의미는 있으나 중요한 의미를 가지고 있지 않다고 판단하여 제거.
            
            2) 이러한 표현들도 전부 emoticon 같은 감정 표현의 의성어로 쓰거나, 단축 표현이므로 제거하지 않음. 
            
        3. http:// 로 시작하는 hyperlink 제거.
        
        4. 특수문자 제거.
    
    """
    # Series의 object를 str로 변경.
    docs = [str(doc) for doc in docs]
    
    # 1. 사진 & 이모티콘 제거 
    pattern1 = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+")
    docs = [pattern1.sub("", doc) for doc in docs]
    
    # 2. 자음으로만 된 텍스트 & 모음으로만 된 텍스트 제거 (필요시)
    pattern2 = re.compile("[ㄱ-ㅎ]*[ㅏ-ㅢ]*")
    docs = [pattern2.sub("", doc) for doc in docs]
    
    # 3. hyperlink 제거 (from. googling)
    # http 로 시작하는 경우 : (https?:\/\/)
    # www.로 시작하는 경우 : ([\w.]+){1,2}
    # www 이후에 .com, .co.kr 등 : (\.[\w]{2,4}){1,2}(.*)
    pattern3 = re.compile(r"\b(https?:\/\/)?([\w.]+){1,2}(\.[\w]{2,4}){1,2}(.*)")
    docs = [pattern3.sub("", doc) for doc in docs]
    
    # 4 특수문자 제거
    pattern4 = re.compile("[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]")
    docs = [pattern4.sub("", doc) for doc in docs]

    return docs

# 불용어 추가
def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    SW.add("있다")
    SW.add("맛있")
    SW.add("좋아")
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    
    with open(path) as f:
        for word in f:
            SW.add(word)
            
    return SW

# 토크나이저 
def text_tokenizing(doc):
    #return [word for word in mecab_pos(doc) if word not in SW and len(word) > 1]
    
    # wordcloud를 위해 명사만 추출하는 경우.
    return [word for word in mecab.nouns(doc) if word not in SW and len(word) > 1]

In [4]:
import pickle
with open("cleaned_review.pk", "rb") as f:
    data = pickle.load(f)
    
data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.info())

EOFError: Ran out of input

In [15]:
corpus = data['r_comments']
corpus

0                                  맛있어요! ㅋㅋ사장님 두분도 친절하세요! 최고
1                                                 맛있고 친절해요!!
2                                                          굿
3                                                          굿
4                                                사장님 친절하세용ㅎㅎ
                                 ...                        
1136974                                                  NaN
1136975                                                  맛나요
1136976    가성비맛집입니다 신선하고 양도 정말많고\n후식으로 양갱하고 요구르트도 주시고\n초밥...
1136977                    초밥도 맛있지만 해물우동 국물은 진짜 딱 칼칼한 꽃게탕이네요
1136978                                                  NaN
Name: r_comments, Length: 1136979, dtype: object

In [16]:
SW = define_stopwords("stopwords-ko.txt")

# 리뷰 텍스트를 정제 (특수문자, 자음&모음 제거, 링크제거, 특수문자 제거)
cleaned_corpus = message_cleaning(corpus)
print(len(cleaned_corpus)) # 정제된 텍스트 길이 확인
print(cleaned_corpus[:10])

1136979
['맛있어요 사장님 두분도 친절하세요 최고', '맛있고 친절해요', '굿', '굿', '사장님 친절하세용', '친절하구 맛있어용', '넘 맛나용', '맛있어요', '너무 맛있어요 ', '항상친절하시고 맛있습니다']


In [17]:
tokenized_data = text_tokenizing(cleaned_corpus)

AttributeError: nouns

In [8]:
tokenized_data = [str(msg).split() for msg in list(cleaned_corpus)]
print(tokenized_data[:10])

[['맛있어요', '사장님', '두분도', '친절하세요', '최고'], ['맛있고', '친절해요'], ['굿'], ['굿'], ['사장님', '친절하세용'], ['친절하구', '맛있어용'], ['넘', '맛나용'], ['맛있어요'], ['너무', '맛있어요'], ['항상친절하시고', '맛있습니다']]


In [9]:
len(tokenized_data)

1136979

# FastText로 유사어 찾기

In [9]:
# gensim에서 FastText 불러오기
from gensim.models import FastText

fast_model = FastText(tokenized_data, min_count=5, sg=1, vector_size=300, workers=4, min_n=2, alpha=0.05, max_n=7, epochs=10, window=7)
fast_model.save('fast_model')

In [10]:
# 모델 확인
# 단어간 유사도 확인
print("맛있어요 와 맜있어용 단어간 유사도는 %.2f 입니다." %(fast_model.wv.similarity('맛있어요','맛있어용')))

맛있어요 와 맜있어용 단어간 유사도는 0.73 입니다.


In [11]:
# 맛있어요 관련된 유사 표현 추출
taste = pd.DataFrame(fast_model.wv.most_similar("맛있어요!",topn=3300), columns=['단어','유사도'])
taste = taste[taste['유사도'] >=0.45]
taste

,단어,유사도
0,맛있어요〰,0.97
1,맛있어요굳,0.97
2,맛있어요1,0.96
3,맛있어요요,0.96
4,맛있어요굳굳,0.95
...,...,...
1157,맛있는빵집,0.45
1158,맛나지요,0.45
1159,자주와여,0.45
1160,대존맛이고,0.45


In [12]:
# 맛있어요와 가장 유사한 단어 100개 추출
# dict만들때 
taste2 = pd.DataFrame(fast_model.wv.most_similar("맛있음",topn=2500), columns=['단어','유사도'])
taste2 = taste2[taste2['유사도'] >=0.3]
taste2

,단어,유사도
0,맛있음다,0.75
1,맛있음♡,0.70
2,맛남,0.69
3,맛있었음,0.68
4,싸고맛있음,0.67
...,...,...
2495,대다나다,0.33
2496,소란스러움,0.33
2497,생각남,0.33
2498,하나도없어요,0.33


In [13]:
# 맛있어요와 가장 유사한 단어 100개 추출
# dict만들때 
taste3 = pd.DataFrame(fast_model.wv.most_similar("맛있다",topn=1000), columns=['단어','유사도'])
taste3 = taste3[taste3['유사도'] >=0.4]
taste3

,단어,유사도
0,맛있다함,0.69
1,맛있음다,0.66
2,맛있었다,0.64
3,맛있다아,0.62
4,좋다,0.62
...,...,...
486,안다,0.40
487,맛있어용❤,0.40
488,낫겠다,0.40
489,보통이다,0.40


In [14]:
# 맛있어요 관련된 유사 표현 추출
taste4 = pd.DataFrame(fast_model.wv.most_similar("맛있어요",topn=3300), columns=['단어','유사도'])
taste4 = taste4[taste4['유사도'] >=0.45]
taste4

,단어,유사도
0,맛있어요요,0.87
1,맛있어요요요,0.84
2,맛있어요맛있어요,0.84
3,맛있어요〰,0.83
4,맛나요,0.83
...,...,...
1362,정말맛있습니다,0.45
1363,신속해요,0.45
1364,꿀맛입니다,0.45
1365,맨날가요,0.45


In [15]:
taste = pd.concat([taste, taste2], axis =0)
taste = pd.concat([taste, taste3], axis =0)
taste = pd.concat([taste, taste4], axis =0)

In [16]:
taste.rename(columns={'단어':'taste'}, inplace=True)
taste_dict = taste.to_dict('list') 
del(taste_dict['유사도'])
taste_dict

{'taste': ['맛있어요〰',
  '맛있어요굳',
  '맛있어요1',
  '맛있어요요',
  '맛있어요굳굳',
  '맛있어요유',
  '맛있어요요용',
  '맛있어요◡̎',
  '맛있어요짱',
  '맛있어요\U0001f973',
  '맛있어요우',
  '맛있어요요요',
  '맛있어요☆',
  '맛있어요대박',
  '맛있어요0',
  '맛있어요굿굿',
  '맛있어요역시',
  '맛있어요정말',
  '맛있어요굿',
  '맛있어요‼️',
  '맛있어요잘',
  '맛있어요제',
  '맛있어요짱짱',
  '맛있어요ᆢ',
  '맛있어요인생',
  '맛있어요✨',
  '맛있어요’',
  '맛있어요\U0001f90e',
  '맛있어요❣',
  '맛있어요꼭',
  '맛있어요요즘',
  '맛있어요전',
  '맛있어요조아요',
  '맛있어요늘',
  '맛있어요〰️',
  '맛있어요⭐️',
  '맛있어요좋아요',
  '맛있어요너무',
  '맛있어요♥︎',
  '맛있어요최고에요',
  '맛있어요☕️',
  '맛있어요오옹',
  '맛있어요많이',
  '맛있어요b',
  '맛있어요♥♥',
  '맛있어요\U0001f972',
  '맛있어요🤗',
  '맛있어요옹',
  '맛있어요\U0001f97a\U0001f97a',
  '맛있어요최고',
  '맛있어요항상',
  '맛있어요완전',
  '맛있어요담엔',
  '맛있어요자주가요',
  '맛있어요……',
  '맛있어요ㆍ',
  '맛있어요〜',
  '맛있어요\U0001f929',
  '맛있어요맛있어요',
  '맛있어요\U0001f9e1',
  '맛있어요\U0001f92d',
  '맛있어요가성비',
  '맛있어요☺',
  '맛있어요',
  '맛있어요D',
  '맛있어요\U0001f9e1\U0001f9e1',
  '맛있어요추천해요',
  '맛있어요\U0001f970\U0001f970\U0001f970',
  '맛있어요오오옹',
  '맛있어요국물이',
  '맛있어요\U0001f970\U0001f970',
  '맛있어요자주',
  '맛있어요깔끔하고'

In [17]:
taste_list = ['맛있어요!','맛있어요','맛있고','맛있게','맛있다','맛있음','맛있어요.']

In [18]:
taste_dict['taste'] = taste_dict['taste'] + taste_list

In [19]:
taste_dict['taste'][::-1]

['맛있어요.',
 '맛있음',
 '맛있다',
 '맛있게',
 '맛있고',
 '맛있어요',
 '맛있어요!',
 '너무맛잇엇어요',
 '맨날가요',
 '꿀맛입니다',
 '신속해요',
 '정말맛있습니다',
 '맛좋음',
 '존맛탱이었어요',
 '굿굿요',
 '맛있어요❤️❤️❤️❤️',
 '꼬소해요',
 '맛있길',
 '멋있네요',
 '짱맛남',
 '존맛입니당',
 '존맛탱이요',
 '맛도좋습니다',
 '깜찍해요',
 '맛잇엇습니당',
 '퀄리티좋아요',
 '멋있어용',
 '좋아요굿',
 '존맛집입니다',
 '마싯어영',
 '맛있답니당',
 '젤맛나요',
 '맛낫어용',
 '커피맛나요',
 '이어요',
 '성지입니다',
 '맛있더라고요',
 '맛이있고',
 '맛집이에유',
 '맛잇었어용',
 '단골각이에요',
 '진하네요',
 '특색있네요',
 '미친놈이에요',
 '정말맛있었습니다',
 '최고요',
 '맛있었어여',
 '묵었어요',
 '최고에요',
 '반해요',
 '맛깔나요',
 '맛나진',
 '많이짜요',
 '맛나는데',
 '한이에요',
 '제일맛있습니다',
 '너무맛있네용',
 '매장깔끔해요',
 '꽂혔네요',
 '아삭해요',
 '비싼값해요',
 '찐하네요',
 '괜찬네요',
 '맛있다하네요',
 '마싯슴',
 '맛탱',
 '정말맛있어여',
 '짱마싯',
 '진짜맛있네요',
 '먹어써요',
 '너무맛있어용♡',
 '넘넘좋아요',
 '맛잇엇구',
 '맛있어봤자',
 '맛좋았습니다',
 '맛이',
 '여유있어요',
 '맛있데서',
 '엄청맛있었어요',
 '너무너무맛있어용',
 '진하구요',
 '먹았어요',
 '맜있었어요',
 '넘맛있는데',
 '좀아요',
 '맛잇게먹고가요',
 '맛있구용',
 '고퀄입니다',
 '시퍼요',
 '맜있습니다',
 '짯네요',
 '새콤했어요',
 '맛도있네요',
 '짭짤해요',
 '혁명이에요',
 '많이줘요',
 '뿜뿜이에요',
 '맛있는데도',
 '너무맛있어여',
 '맛있는편이에요',
 '최애예요',
 '맛있지만요',
 '안차요',
 

In [20]:
# 전체 dict에 추가
keyword_dict ={}
keyword_dict.update(taste_dict)
keyword_dict

{'taste': ['맛있어요〰',
  '맛있어요굳',
  '맛있어요1',
  '맛있어요요',
  '맛있어요굳굳',
  '맛있어요유',
  '맛있어요요용',
  '맛있어요◡̎',
  '맛있어요짱',
  '맛있어요\U0001f973',
  '맛있어요우',
  '맛있어요요요',
  '맛있어요☆',
  '맛있어요대박',
  '맛있어요0',
  '맛있어요굿굿',
  '맛있어요역시',
  '맛있어요정말',
  '맛있어요굿',
  '맛있어요‼️',
  '맛있어요잘',
  '맛있어요제',
  '맛있어요짱짱',
  '맛있어요ᆢ',
  '맛있어요인생',
  '맛있어요✨',
  '맛있어요’',
  '맛있어요\U0001f90e',
  '맛있어요❣',
  '맛있어요꼭',
  '맛있어요요즘',
  '맛있어요전',
  '맛있어요조아요',
  '맛있어요늘',
  '맛있어요〰️',
  '맛있어요⭐️',
  '맛있어요좋아요',
  '맛있어요너무',
  '맛있어요♥︎',
  '맛있어요최고에요',
  '맛있어요☕️',
  '맛있어요오옹',
  '맛있어요많이',
  '맛있어요b',
  '맛있어요♥♥',
  '맛있어요\U0001f972',
  '맛있어요🤗',
  '맛있어요옹',
  '맛있어요\U0001f97a\U0001f97a',
  '맛있어요최고',
  '맛있어요항상',
  '맛있어요완전',
  '맛있어요담엔',
  '맛있어요자주가요',
  '맛있어요……',
  '맛있어요ㆍ',
  '맛있어요〜',
  '맛있어요\U0001f929',
  '맛있어요맛있어요',
  '맛있어요\U0001f9e1',
  '맛있어요\U0001f92d',
  '맛있어요가성비',
  '맛있어요☺',
  '맛있어요',
  '맛있어요D',
  '맛있어요\U0001f9e1\U0001f9e1',
  '맛있어요추천해요',
  '맛있어요\U0001f970\U0001f970\U0001f970',
  '맛있어요오오옹',
  '맛있어요국물이',
  '맛있어요\U0001f970\U0001f970',
  '맛있어요자주',
  '맛있어요깔끔하고'

In [21]:
# 친절해요 와 가장 유사한 단어 
kindness = pd.DataFrame(fast_model.wv.most_similar("친절",topn=1000), columns=['단어','유사도'])
kindness = kindness[kindness['유사도'] >=0.35]
kindness

,단어,유사도
0,친절행,0.67
1,친절굿,0.67
2,친절요,0.65
3,맛친절,0.64
4,친절스,0.64
...,...,...
424,한옥분위기,0.35
425,미모,0.35
426,친절한지,0.35
427,친절했는데,0.35


In [22]:
# 친절해요 와 가장 유사한 단어 
kindness2 = pd.DataFrame(fast_model.wv.most_similar("서비스",topn=700), columns=['단어','유사도'])
kindness2 = kindness2[kindness2['유사도'] >=0.4]
kindness2


,단어,유사도
0,서비스굳,0.71
1,응대서비스,0.69
2,서비스응대,0.68
3,친절서비스,0.66
4,서비스맛,0.64
...,...,...
204,서비스업,0.40
205,안내까지,0.40
206,서비스인데도,0.40
207,코스구성,0.40


In [23]:
kindness = pd.concat([kindness, kindness2], axis =0)

In [24]:
kindness.rename(columns={'단어':'kindness'}, inplace=True)
kindness_dict = kindness.to_dict('list') 
del(kindness_dict['유사도'])
kindness_dict

{'kindness': ['친절행',
  '친절굿',
  '친절요',
  '맛친절',
  '친절스',
  '친절해료',
  '친절들',
  '개친절',
  '친절해요ㆍ',
  '친절해영',
  '친절해요오',
  '친절하도',
  '친절해욧',
  '친절하시어',
  '친절해오',
  '친절허시고',
  '노친절',
  '친절만',
  '친절하시네오',
  '맛있고친절',
  '친절맛',
  '직원친절',
  '친철',
  '친절하시규',
  '친절짱',
  '친절하공',
  '친절♡',
  '친절하시고',
  '친절도',
  '친절하심요',
  '친절해유',
  '친절하시기',
  '세상친절',
  '친절시고',
  '친절하규',
  '친절하시고여',
  '친절해요\U0001f970',
  '친절하세요',
  '친절하셨오요',
  '친절햐요',
  '친절하셧고',
  '친절하세요🤗',
  '친절하세요\U0001f9e1',
  '친절하심♡',
  '친절해요친절해요',
  '친절쓰',
  '친절하서',
  '친절하시거',
  '친절함',
  '친절하시',
  '핵친절',
  '서비스친절',
  '친절하세오',
  '친절서비스',
  '친절하시네여',
  '친절함요',
  '친절해요',
  '친절하세',
  '친절하시구오',
  '친절하고',
  '친절하요',
  '친절하세요오',
  '친절하시며',
  '친절하세요또',
  '친절하세욥',
  '친절하세유',
  '친절하시고좋아요',
  '친절하실',
  '매우친절',
  '친절하시도',
  '친절하시고용',
  '친절히고',
  '친절하심',
  '친절하',
  '친절해요♡',
  '친절하세영',
  '친절하시기는',
  '친절하네여',
  '친절하셔유',
  '친절하시기도',
  '친절하시네요',
  '친절햇고',
  '친절해요♥️',
  '친절해욥',
  '친절해욤',
  '친절하고좋습니다',
  '완전친절',
  '친절하세요♥️',
  '친절이나',
  '친절하십니다',
  '친절하기고',
  '매우불친절

In [25]:
kindness_list = ['친절','서비스']
kindness_dict['kindness'] = kindness_dict['kindness'] + kindness_list

In [26]:
keyword_dict.update(kindness_dict)
keyword_dict

{'taste': ['맛있어요〰',
  '맛있어요굳',
  '맛있어요1',
  '맛있어요요',
  '맛있어요굳굳',
  '맛있어요유',
  '맛있어요요용',
  '맛있어요◡̎',
  '맛있어요짱',
  '맛있어요\U0001f973',
  '맛있어요우',
  '맛있어요요요',
  '맛있어요☆',
  '맛있어요대박',
  '맛있어요0',
  '맛있어요굿굿',
  '맛있어요역시',
  '맛있어요정말',
  '맛있어요굿',
  '맛있어요‼️',
  '맛있어요잘',
  '맛있어요제',
  '맛있어요짱짱',
  '맛있어요ᆢ',
  '맛있어요인생',
  '맛있어요✨',
  '맛있어요’',
  '맛있어요\U0001f90e',
  '맛있어요❣',
  '맛있어요꼭',
  '맛있어요요즘',
  '맛있어요전',
  '맛있어요조아요',
  '맛있어요늘',
  '맛있어요〰️',
  '맛있어요⭐️',
  '맛있어요좋아요',
  '맛있어요너무',
  '맛있어요♥︎',
  '맛있어요최고에요',
  '맛있어요☕️',
  '맛있어요오옹',
  '맛있어요많이',
  '맛있어요b',
  '맛있어요♥♥',
  '맛있어요\U0001f972',
  '맛있어요🤗',
  '맛있어요옹',
  '맛있어요\U0001f97a\U0001f97a',
  '맛있어요최고',
  '맛있어요항상',
  '맛있어요완전',
  '맛있어요담엔',
  '맛있어요자주가요',
  '맛있어요……',
  '맛있어요ㆍ',
  '맛있어요〜',
  '맛있어요\U0001f929',
  '맛있어요맛있어요',
  '맛있어요\U0001f9e1',
  '맛있어요\U0001f92d',
  '맛있어요가성비',
  '맛있어요☺',
  '맛있어요',
  '맛있어요D',
  '맛있어요\U0001f9e1\U0001f9e1',
  '맛있어요추천해요',
  '맛있어요\U0001f970\U0001f970\U0001f970',
  '맛있어요오오옹',
  '맛있어요국물이',
  '맛있어요\U0001f970\U0001f970',
  '맛있어요자주',
  '맛있어요깔끔하고'

In [27]:
mood = pd.DataFrame(fast_model.wv.most_similar("분위기",topn=2000), columns=['단어','유사도'])
mood = mood[mood['유사도'] >=0.4]
mood

,단어,유사도
0,분위기굳,0.81
1,분위기넘,0.78
2,분위기조,0.77
3,분위기도,0.76
4,분위기더,0.75
...,...,...
502,야경이,0.40
503,아늑했습니다,0.40
504,가구들이,0.40
505,소개팅하기도,0.40


In [28]:
mood2 = pd.DataFrame(fast_model.wv.most_similar("감성",topn=2000), columns=['단어','유사도'])
mood2 = mood2[mood2['유사도'] >=0.35]
mood2

,단어,유사도
0,레트로감성,0.64
1,감성은,0.63
2,감성돋는,0.62
3,감성만,0.62
4,감성의,0.61
...,...,...
498,한국적,0.35
499,한성대입구역,0.35
500,데클렌커피,0.35
501,디저트가게,0.35


In [29]:
mood3 = pd.DataFrame(fast_model.wv.most_similar("인테리어",topn=2000), columns=['단어','유사도'])
mood3 = mood3[mood3['유사도'] >=0.35]
mood3

,단어,유사도
0,분위기인테리어,0.77
1,인테리어굿,0.76
2,인테리어고,0.74
3,인테리어예뻐요,0.73
4,인테리어인데,0.72
...,...,...
1663,분의기는,0.35
1664,눈과입이,0.35
1665,단정한,0.35
1666,구석이,0.35


In [30]:
mood = pd.concat([mood, mood2], axis =0)
mood = pd.concat([mood, mood3], axis =0)

In [31]:
mood.rename(columns={'단어':'mood'}, inplace=True)
mood_dict = mood.to_dict('list') 
del(mood_dict['유사도'])

In [32]:
mood_list = ['분위기','감성','인테리어']
mood_dict['mood'] = mood_dict['mood'] + mood_list

In [33]:
keyword_dict.update(mood_dict)
keyword_dict

{'taste': ['맛있어요〰',
  '맛있어요굳',
  '맛있어요1',
  '맛있어요요',
  '맛있어요굳굳',
  '맛있어요유',
  '맛있어요요용',
  '맛있어요◡̎',
  '맛있어요짱',
  '맛있어요\U0001f973',
  '맛있어요우',
  '맛있어요요요',
  '맛있어요☆',
  '맛있어요대박',
  '맛있어요0',
  '맛있어요굿굿',
  '맛있어요역시',
  '맛있어요정말',
  '맛있어요굿',
  '맛있어요‼️',
  '맛있어요잘',
  '맛있어요제',
  '맛있어요짱짱',
  '맛있어요ᆢ',
  '맛있어요인생',
  '맛있어요✨',
  '맛있어요’',
  '맛있어요\U0001f90e',
  '맛있어요❣',
  '맛있어요꼭',
  '맛있어요요즘',
  '맛있어요전',
  '맛있어요조아요',
  '맛있어요늘',
  '맛있어요〰️',
  '맛있어요⭐️',
  '맛있어요좋아요',
  '맛있어요너무',
  '맛있어요♥︎',
  '맛있어요최고에요',
  '맛있어요☕️',
  '맛있어요오옹',
  '맛있어요많이',
  '맛있어요b',
  '맛있어요♥♥',
  '맛있어요\U0001f972',
  '맛있어요🤗',
  '맛있어요옹',
  '맛있어요\U0001f97a\U0001f97a',
  '맛있어요최고',
  '맛있어요항상',
  '맛있어요완전',
  '맛있어요담엔',
  '맛있어요자주가요',
  '맛있어요……',
  '맛있어요ㆍ',
  '맛있어요〜',
  '맛있어요\U0001f929',
  '맛있어요맛있어요',
  '맛있어요\U0001f9e1',
  '맛있어요\U0001f92d',
  '맛있어요가성비',
  '맛있어요☺',
  '맛있어요',
  '맛있어요D',
  '맛있어요\U0001f9e1\U0001f9e1',
  '맛있어요추천해요',
  '맛있어요\U0001f970\U0001f970\U0001f970',
  '맛있어요오오옹',
  '맛있어요국물이',
  '맛있어요\U0001f970\U0001f970',
  '맛있어요자주',
  '맛있어요깔끔하고'

In [34]:
# 가격 부정 
neg_price = pd.DataFrame(fast_model.wv.most_similar("비싸",topn=2000), columns=['단어','유사도'])
neg_price = neg_price[neg_price['유사도'] >=0.3]
neg_price

,단어,유사도
0,비싸당,0.73
1,비싸네,0.68
2,비싸뇨,0.68
3,비싸유,0.68
4,비싸욥,0.67
...,...,...
801,그것땜에,0.30
802,45000원,0.30
803,너무추워요,0.30
804,26000원,0.30


In [35]:
# 가격 부정 
neg_price2 = pd.DataFrame(fast_model.wv.most_similar("너무비싸",topn=2000), columns=['단어','유사도'])
neg_price2 = neg_price2[neg_price2['유사도'] >=0.5]
neg_price2

,단어,유사도
0,너무비싸고,0.92
1,너무비싸요,0.90
2,너무비싸서,0.89
3,너무비싸네요,0.89
4,너무비쌈,0.80
...,...,...
204,재방문의사없어요,0.50
205,다른지점에비해,0.50
206,너무많음,0.50
207,변한거같아요,0.50


In [36]:
neg_price = pd.concat([neg_price, neg_price2], axis =0)

In [37]:
neg_price.rename(columns={'단어':'neg_price'}, inplace=True)
neg_price_dict = neg_price.to_dict('list') 
del(neg_price_dict['유사도'])

In [38]:
neg_price_list = ['비싸','너무']
neg_price_dict['neg_price'] = neg_price_dict['neg_price'] + neg_price_list

In [39]:
keyword_dict.update(neg_price_dict)
keyword_dict

{'taste': ['맛있어요〰',
  '맛있어요굳',
  '맛있어요1',
  '맛있어요요',
  '맛있어요굳굳',
  '맛있어요유',
  '맛있어요요용',
  '맛있어요◡̎',
  '맛있어요짱',
  '맛있어요\U0001f973',
  '맛있어요우',
  '맛있어요요요',
  '맛있어요☆',
  '맛있어요대박',
  '맛있어요0',
  '맛있어요굿굿',
  '맛있어요역시',
  '맛있어요정말',
  '맛있어요굿',
  '맛있어요‼️',
  '맛있어요잘',
  '맛있어요제',
  '맛있어요짱짱',
  '맛있어요ᆢ',
  '맛있어요인생',
  '맛있어요✨',
  '맛있어요’',
  '맛있어요\U0001f90e',
  '맛있어요❣',
  '맛있어요꼭',
  '맛있어요요즘',
  '맛있어요전',
  '맛있어요조아요',
  '맛있어요늘',
  '맛있어요〰️',
  '맛있어요⭐️',
  '맛있어요좋아요',
  '맛있어요너무',
  '맛있어요♥︎',
  '맛있어요최고에요',
  '맛있어요☕️',
  '맛있어요오옹',
  '맛있어요많이',
  '맛있어요b',
  '맛있어요♥♥',
  '맛있어요\U0001f972',
  '맛있어요🤗',
  '맛있어요옹',
  '맛있어요\U0001f97a\U0001f97a',
  '맛있어요최고',
  '맛있어요항상',
  '맛있어요완전',
  '맛있어요담엔',
  '맛있어요자주가요',
  '맛있어요……',
  '맛있어요ㆍ',
  '맛있어요〜',
  '맛있어요\U0001f929',
  '맛있어요맛있어요',
  '맛있어요\U0001f9e1',
  '맛있어요\U0001f92d',
  '맛있어요가성비',
  '맛있어요☺',
  '맛있어요',
  '맛있어요D',
  '맛있어요\U0001f9e1\U0001f9e1',
  '맛있어요추천해요',
  '맛있어요\U0001f970\U0001f970\U0001f970',
  '맛있어요오오옹',
  '맛있어요국물이',
  '맛있어요\U0001f970\U0001f970',
  '맛있어요자주',
  '맛있어요깔끔하고'

In [40]:
# 가격 긍정
pos_price = pd.DataFrame(fast_model.wv.most_similar("혜자",topn=50), columns=['단어','유사도'])
pos_price = pos_price[pos_price['유사도'] >=0.4]
pos_price

,단어,유사도
0,혜자혜자,0.80
1,혜자임,0.71
2,혜자인,0.67
3,혜자에,0.66
4,혜자라,0.66
5,혜자가,0.64
6,혜자롭고,0.63
7,혜자고,0.60
8,혜자에요,0.59
9,혜자로운,0.59


In [41]:
# 가격 긍정
pos_price2 = pd.DataFrame(fast_model.wv.most_similar("가성비",topn=1000), columns=['단어','유사도'])
pos_price2 = pos_price2[pos_price2['유사도'] >=0.5]
pos_price2

,단어,유사도
0,가성비가심비,0.74
1,가성비값,0.70
2,가성비킹,0.70
3,가성,0.68
4,가성비쵝오,0.66
5,가성비굳,0.66
6,가성비도,0.65
7,가성비맛,0.65
8,맛가성비,0.65
9,가성비가,0.63


In [42]:
# 가격 긍정
pos_price3 = pd.DataFrame(fast_model.wv.most_similar("합리적",topn=1000), columns=['단어','유사도'])
pos_price3 = pos_price3[pos_price3['유사도'] >=0.35]
pos_price3

,단어,유사도
0,합리적임,0.77
1,합리적가격,0.73
2,합리적이다,0.67
3,합리적으로,0.66
4,합리적이구요,0.65
...,...,...
106,착했어요,0.35
107,구성까지,0.35
108,균일한,0.35
109,저렴한것같아요,0.35


In [43]:
pos_price = pd.concat([pos_price, pos_price2], axis =0)
pos_price = pd.concat([pos_price, pos_price3], axis =0)

In [44]:
pos_price.rename(columns={'단어':'pos_price'}, inplace=True)
pos_price_dict = pos_price.to_dict('list') 
del(pos_price_dict['유사도'])

In [45]:
pos_price_list = ['혜자','가성비','합리적']
pos_price_dict['pos_price'] = pos_price_dict['pos_price'] + pos_price_list

In [46]:
keyword_dict.update(pos_price_dict)
keyword_dict

{'taste': ['맛있어요〰',
  '맛있어요굳',
  '맛있어요1',
  '맛있어요요',
  '맛있어요굳굳',
  '맛있어요유',
  '맛있어요요용',
  '맛있어요◡̎',
  '맛있어요짱',
  '맛있어요\U0001f973',
  '맛있어요우',
  '맛있어요요요',
  '맛있어요☆',
  '맛있어요대박',
  '맛있어요0',
  '맛있어요굿굿',
  '맛있어요역시',
  '맛있어요정말',
  '맛있어요굿',
  '맛있어요‼️',
  '맛있어요잘',
  '맛있어요제',
  '맛있어요짱짱',
  '맛있어요ᆢ',
  '맛있어요인생',
  '맛있어요✨',
  '맛있어요’',
  '맛있어요\U0001f90e',
  '맛있어요❣',
  '맛있어요꼭',
  '맛있어요요즘',
  '맛있어요전',
  '맛있어요조아요',
  '맛있어요늘',
  '맛있어요〰️',
  '맛있어요⭐️',
  '맛있어요좋아요',
  '맛있어요너무',
  '맛있어요♥︎',
  '맛있어요최고에요',
  '맛있어요☕️',
  '맛있어요오옹',
  '맛있어요많이',
  '맛있어요b',
  '맛있어요♥♥',
  '맛있어요\U0001f972',
  '맛있어요🤗',
  '맛있어요옹',
  '맛있어요\U0001f97a\U0001f97a',
  '맛있어요최고',
  '맛있어요항상',
  '맛있어요완전',
  '맛있어요담엔',
  '맛있어요자주가요',
  '맛있어요……',
  '맛있어요ㆍ',
  '맛있어요〜',
  '맛있어요\U0001f929',
  '맛있어요맛있어요',
  '맛있어요\U0001f9e1',
  '맛있어요\U0001f92d',
  '맛있어요가성비',
  '맛있어요☺',
  '맛있어요',
  '맛있어요D',
  '맛있어요\U0001f9e1\U0001f9e1',
  '맛있어요추천해요',
  '맛있어요\U0001f970\U0001f970\U0001f970',
  '맛있어요오오옹',
  '맛있어요국물이',
  '맛있어요\U0001f970\U0001f970',
  '맛있어요자주',
  '맛있어요깔끔하고'

In [47]:
# load json module
import json



# create json object from dictionary
json = json.dumps(keyword_dict)

# open file for writing, "w" 
f = open("keyword_dict.json","w")

# write json object to file
f.write(json)

# close file
f.close()

# kye값별 상위 50개 유사도 테이블 만들기

In [76]:
taste_df = taste.iloc[:30,] # 맛 리스트
kindness_df = kindness.iloc[:30,] # 친절 리스트
mood_df = mood.iloc[:30,] # 분위기 리스트
neg_price_df = neg_price.iloc[:30,] # 가격 긍정 리스트
pos_price_df = pos_price.iloc[:30,] # 가격 부정 리스트

In [78]:
taste_df.to_csv('맛집top30.csv', encoding='utf-8')
kindness_df.to_csv('친절top30.csv', encoding='utf-8')
mood_df.to_csv('분위기top30.csv', encoding='utf-8')
neg_price_df.to_csv('가격부정top30.csv', encoding='utf-8')
pos_price_df.to_csv('가격긍정top30.csv', encoding='utf-8')

In [88]:
pd.merge(taste_df, kindness_df)

,taste,유사도
0,맛있어요〰,0.97
1,맛있어요굳,0.97
2,맛있어요1,0.96
3,맛있어요요,0.96
4,맛있어요굳굳,0.95
5,맛있어요유,0.95
6,맛있어요요용,0.95
7,맛있어요◡̎,0.95
8,맛있어요짱,0.94
9,맛있어요🥳,0.94


In [80]:
type(taste_df)

pandas.core.frame.DataFrame

In [81]:
type(kindness_df)

pandas.core.frame.DataFrame

In [84]:
type(neg_price_df)

pandas.core.frame.DataFrame

In [83]:
type(pos_price_df)

pandas.core.frame.DataFrame

In [79]:
pd.concat(['taste_df','kindness_df'], ignore_index=True, axis=1)

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [ ]:
import seaborn as sns
sns.set(style='whitegrid', font = 'KoPubDotum' , font_scale=1.3)
g = sns.Pair Grid(df, x_vars = df